In [17]:
import pandas as pd
import requests
from googletrans import Translator

In [18]:
data = pd.read_csv('/home/anais/code/anaisdangeot/mood_detector/raw_data/dataset.csv')
data.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

In [4]:
data['track_id'].isnull().value_counts()

track_id
False    114000
Name: count, dtype: int64

In [36]:
data_subsample = data.sample(50)

In [37]:
base_url = 'https://spotify-lyric-api.herokuapp.com/'

def request_lyrics(track_id):
    params = {'trackid': track_id}
    response = requests.get(base_url, params=params).json()
    trans = Translator()
    try:
        song_det = response['lines']
        lyrics = []
        for i in range(len(song_det)):
            lyrics.append(song_det[i]['words'])
            lyrics_clean= (' '.join(lyrics))
        if trans.detect(lyrics_clean).lang != "en":
            lyrics_clean = trans.translate(lyrics_clean, dest='en').text
        return lyrics_clean
    except:
        return 999


data_subsample['lyrics'] = data_subsample['track_id'].map(lambda x: request_lyrics(x))
data_subsample['lyrics']

40983                                                   999
80715                                                   999
3417      Somebody once told me The world is gonna roll ...
108830                                                  999
81218     Desi Crew, Desi Crew CreW CreW, Desi Crew, Stu...
15177     It happened so fast, crash Hurts so good, I ca...
83695                                                   999
37662                                                   999
63820     I want to deliver this song to you Melody This...
77784                                                   999
34398     Ooh child Things are gonna get easier Ooh chil...
113800    Dear God I've been trying awful hard to make y...
23050     Got ready for the night and She's heading for ...
86431                                                   999
24250     Close the doors to temptation Open wide, the g...
25571     I remember all the cries It's a secret dream I...
40608     Those who see the portrait and

In [38]:
data_subsample[data_subsample['lyrics'] != 999]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,lyrics
3417,3417,2s0QwtFt0lBDGPCnJeDvJz,Smash Mouth,Halloween 2022,All Star,0,200373,False,0.731,0.867,...,1,0.0320,0.039400,0.000000,0.0861,0.776,104.020,4,alternative,Somebody once told me The world is gonna roll ...
81218,81218,4XxfOvudrnBRdlgzEaq1sd,Diljit Dosanjh,G.O.A.T.,Born to Shine,69,213333,False,0.745,0.826,...,0,0.0866,0.357000,0.000000,0.3020,0.674,90.149,4,pop,"Desi Crew, Desi Crew CreW CreW, Desi Crew, Stu..."
15177,15177,2K2Fn8vJzgGUG9JMmPFYC3,Mokita;Charlotte Sands,Crash,Crash,63,168274,False,0.696,0.678,...,0,0.0391,0.441000,0.000000,0.0690,0.517,146.998,4,chill,"It happened so fast, crash Hurts so good, I ca..."
63820,63820,1AIayie049DuB4oClgKJ0A,MONGOL800;WANIMA,愛彌々,愛彌々,32,308809,False,0.319,0.875,...,1,0.0819,0.007870,0.000000,0.2990,0.584,84.570,4,j-rock,I want to deliver this song to you Melody This...
34398,34398,7DmZ1e9hh9CUX0PZzZA25C,Dusty Springfield,Classic Christmas Music 2022,O Holy Child,0,144893,False,0.434,0.360,...,0,0.0268,0.912000,0.000000,0.1660,0.439,81.523,4,folk,Ooh child Things are gonna get easier Ooh chil...
113800,113800,4qCv9ASsTJ4kg8gsUWUaAD,Cory Asbury;Anna Asbury,Dear God (Acoustic),Dear God - Acoustic,40,217373,False,0.626,0.195,...,1,0.0379,0.876000,0.000000,0.1170,0.532,153.712,4,world-music,Dear God I've been trying awful hard to make y...
23050,23050,1HrMWH5GUdK6Yi94rbANJA,CamelPhat;Elderbrook,Cola,Cola,72,223869,False,0.706,0.740,...,1,0.0325,0.024500,0.512000,0.1350,0.444,122.007,3,deep-house,Got ready for the night and She's heading for ...
24250,24250,5YdfCDRO1rF4Fg0fufm5iU,Octave One;Ann Saunderson,One Black Water (feat. Ann Saunderson) [Full S...,Blackwater - radio edit 128 strings mix,37,222187,False,0.743,0.826,...,0,0.0650,0.212000,0.000021,0.1620,0.558,128.236,4,detroit-techno,"Close the doors to temptation Open wide, the g..."
25571,25571,3HWaSYU8Q3XNuJlF6nLrfq,Silent Circle,No.1 (Jubiläums Edition),Touch in the Night,55,319813,False,0.744,0.812,...,1,0.0379,0.037500,0.013800,0.1080,0.865,122.083,4,disco,I remember all the cries It's a secret dream I...
40608,40608,6jzcEtCzGYnslrheybOMoc,Oficina G3,Retrato,Retrato,40,333250,False,0.498,0.768,...,0,0.0317,0.006250,0.001520,0.6470,0.148,139.964,4,gospel,Those who see the portrait and the wide smile ...


In [24]:
data_subsample['lyrics'].value_counts()

lyrics
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
data['lyrics'].value_counts()

In [ ]:
data.to_csv('/home/anais/code/anaisdangeot/mood_detector/raw_data/dataset_added_lyrics.csv')